# autopilotで作成されたエンドポイントに対して推論を行う


1) sagemaker.predict()を使う


2) envoke_endpoint()を使う

In [1]:
import pandas as pd

# 推論データ読み込み

In [2]:
df_inf = pd.read_csv('../input_for_autopilot/test.csv')

In [3]:
df_inf

,date,store_nbr,family,onpromotion,avg_oil,wd,dofw_1,dofw_2,dofw_3,dofw_4,dofw_5,dofw_6,type_Additional,type_Event,type_Holiday,school_season
0,2017-08-16,1,AUTOMOTIVE,0,48.281429,1,0,1,0,0,0,0,0,0,0,1
1,2017-08-16,1,BABY CARE,0,48.281429,1,0,1,0,0,0,0,0,0,0,1
2,2017-08-16,1,BEAUTY,2,48.281429,1,0,1,0,0,0,0,0,0,0,1
3,2017-08-16,1,BEVERAGES,20,48.281429,1,0,1,0,0,0,0,0,0,0,1
4,2017-08-16,1,BOOKS,0,48.281429,1,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,2017-08-31,9,POULTRY,1,47.060000,1,0,0,1,0,0,0,0,0,0,1
28508,2017-08-31,9,PREPARED FOODS,0,47.060000,1,0,0,1,0,0,0,0,0,0,1
28509,2017-08-31,9,PRODUCE,1,47.060000,1,0,0,1,0,0,0,0,0,0,1
28510,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.060000,1,0,0,1,0,0,0,0,0,0,1


## 1) 既存のエンドポイントを指定して、predict()を実施する : SageMaker SDK

https://aws.amazon.com/jp/blogs/news/anomaly-detection-on-amazon-dynamodb-streams-using-the-amazon-sagemaker-random-cut-forest-algorithm/

In [ ]:
import boto3

In [ ]:
import sagemaker

In [ ]:
cli_sagemaker = boto3.client('sagemaker')

In [ ]:
xgb_predictor = sagemaker.predictor.Predictor('kaggle-sales-tgt-rlt',
                              sagemaker_session=sagemaker.Session(), 
                              serializer=sagemaker.serializers.CSVSerializer(), 
                              deserializer=sagemaker.deserializers.JSONDeserializer())

In [ ]:
xgb_predictor.predict()

# 2)エンドポイントにリクエストを送る : boto3

https://dev.classmethod.jp/articles/amazon-sagemaker-predict-api-boto3/

In [5]:
import boto3
client = boto3.client('sagemaker-runtime')

In [6]:
import numpy as np

シリアライザを定義

https://github.com/aws-samples/amazon-sagemaker-examples-jp/blob/master/xgboost_customer_churn/xgboost_customer_churn.ipynb

In [ ]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
df_inf.loc[1,:].to_csv()

In [ ]:
np.array(df_inf.loc[1,:]).to_csv()

## dfから行を切り取って、csv形式のバイトデータにする

In [ ]:
### ゴール
b'2017-08-16,1,AUTOMOTIVE,0,48.281429,1,0,1,0,0,0,0,0,0,0,1'

In [ ]:
df_inf.loc[1,:].to_json()

In [ ]:
df_inf.loc[1,:].to_csv(index=False,header=False).replace('\n',',')[:-1]

In [ ]:
response = client.invoke_endpoint(
    EndpointName='kaggle-sales-tgt-rlt',
    #Body=b'2017-08-16,1,AUTOMOTIVE,0,48.281429,1,0,1,0,0,0,0,0,0,0,1',
    #Body='2017-08-16,1,AUTOMOTIVE,0,48.281429,1,0,1,0,0,0,0,0,0,0,1',
    Body=df_inf.loc[1,:].to_csv(index=False,header=False).replace('\n',',')[:-1],
    ContentType='text/csv',
    Accept='application/json'
    #Accept='text/csv'
)

In [ ]:
response

In [ ]:
import json
my_json = json.loads(response['Body'].read())

In [ ]:
my_json

In [ ]:
l = []
l.append(my_json)

In [ ]:
l.append(my_json)

In [ ]:
l

In [ ]:
df_inf

In [ ]:
pd.concat([df_inf[['date', 'store_nbr', 'family']], pd.DataFrame(l, columns=['sales'])], axis=1)

In [7]:
import json

l = []
for i in range(28512):
    response = client.invoke_endpoint(
        EndpointName='kaggle-sales-tgt-rlt',
        #Body=b'2017-08-16,1,AUTOMOTIVE,0,48.281429,1,0,1,0,0,0,0,0,0,0,1',
        #Body='2017-08-16,1,AUTOMOTIVE,0,48.281429,1,0,1,0,0,0,0,0,0,0,1',
        Body=df_inf.loc[i,:].to_csv(index=False,header=False).replace('\n',',')[:-1],
        ContentType='text/csv',
        Accept='application/json'
        #Accept='text/csv'
    )
    my_json = json.loads(response['Body'].read())
    l.append(my_json)

df_pred = pd.concat([df_inf[['date', 'store_nbr', 'family']], pd.DataFrame(l, columns=['sales'])], axis=1)

In [8]:
df_pred.to_csv('../tmp_df_pred.csv')

In [9]:
df_pred

,date,store_nbr,family,sales
0,2017-08-16,1,AUTOMOTIVE,5.225559
1,2017-08-16,1,BABY CARE,-0.292743
2,2017-08-16,1,BEAUTY,33.668690
3,2017-08-16,1,BEVERAGES,2298.900635
4,2017-08-16,1,BOOKS,2.162561
...,...,...,...,...
28507,2017-08-31,9,POULTRY,340.854858
28508,2017-08-31,9,PREPARED FOODS,74.030815
28509,2017-08-31,9,PRODUCE,1072.338745
28510,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,-37.679092


# 提出ファイル作成

In [10]:
df_test = pd.read_csv('../input/test.csv')

In [11]:
df_test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [14]:
df_mrg = pd.merge(df_test, df_pred, how='left', on=['date', 'store_nbr', 'family'])

In [15]:
df_mrg

,id,date,store_nbr,family,onpromotion,sales
0,3000888,2017-08-16,1,AUTOMOTIVE,0,5.225559
1,3000889,2017-08-16,1,BABY CARE,0,-0.292743
2,3000890,2017-08-16,1,BEAUTY,2,33.668690
3,3000891,2017-08-16,1,BEVERAGES,20,2298.900635
4,3000892,2017-08-16,1,BOOKS,0,2.162561
...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,340.854858
28508,3029396,2017-08-31,9,PREPARED FOODS,0,74.030815
28509,3029397,2017-08-31,9,PRODUCE,1,1072.338745
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,-37.679092


In [16]:
df_mrg[['id' ,'sales']].to_csv('../submit/sub_autopilot.csv', index=False)